# Predicting from Sound
> A Neural Networks project by Aleksander Nikolajev, Kayahan Kaya and Severin Brunner

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- sticky_rank: 1

## Introduction
The objective of this project was to find ways of obtaining information from sounds using neural networks. In particular, we are trying to predict the source of a sound as well as the distance between source and microphone given a specific audio sample.
To approach the detection of the sound source, we are constructing a neural network based on a submission {% fn 1 %}
to the [Freesound General-Purpose Audio Tagging Challenge](https://www.kaggle.com/c/freesound-audio-tagging/overview), a competition with the goal of classifying audio files from a wide range of real-world environments.


## Audio theory
In this section, essential theoretical elements of audio analysis are introduced.

### Pulse-code modulation (PCM)
In order to store an analog audio signal in memory, it has to be digitized by applying sampling and quantization. Sampling refers to measuring the signal values at specific timesteps, which produces a time-discrete signal from an time-continuous signal. Quantization means mapping the continuous signal values to discrete values in a specific range, e.g. 16 bits.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Pcm.svg/1024px-Pcm.svg.png "Sampling and quantization of an analog signal (red) with 4-bit PCM, resulting in a time-discrete and value-discrete signal (blue) it can be seen, only a part of the samples has been manually verified (blue).{% fn 2 %}")

PCM is a format that stores uncompressed audio signals by simply storing an array of values that have been produced by sampling and quantizing an analog signal. It has two basic properties:  The sampling rate (how many samples per second were taken) and the bit depth (how many bits each sample value has), which determines the resolution. A typical sampling rate is 44.1 kHz (e.g. CDs), and 16 bits is a common choice for the bit depth. The data and sampling rate can be read from an audio file in wav format with scipy as follows:



In [ ]:
import matplotlib.pyplot as plt
from scipy.io import wavfile

rate, data = wavfile.read('audiofile.wav')
print("Sampling rate:", rate)
# if bit depth is 16, data is stored as a numpy int16 array
print("Data:", data)
plt.plot(data)

### Spectograms

A spectrogram is a visualiziaton of the frequency spectrum of a signal over time. The frequency spectrum basically represents the signal strength of various frequencies present in the signal. It is depicted as a heat map, which means the intensity at a specific frequency and time is shown by varying the color.
TODO: add example Spectogram

TODO: Features to extract from an audio signal
spectral features (frequency-based), e.g. spectral centroid, spectral density ...

### MFCC (Mel-Frequency Cepstral Coefficients)
The MFCCs of a signal are a small set of features (10-20) that concisely describe the overall shape of a spectrum.


## The Dataset

As a dataset for testing and training, we are using the one provided for the 2018 Kaggle Freesound competition, which is downloadable [here](https://www.kaggle.com/c/freesound-audio-tagging/data).
It contains sounds from 41 different categories such as trumpet or fireworks, with 9473 training examples and 1600 test examples. However the samples aren't distributed uniformly over the categories, meaning there's more data for some categories than for others. Also, the amount of manually verified samples varies from category to category.  This might cause the training to become more challenging.

![](kaggle_dataset_distribution.png "The dataset. As it can be seen, only a part of the samples has been manually verified (blue). The amount of samples and the fraction of manually verified samples per category varies between the categories.")

## Creating the network

We are using Keras as the deep learning library to construct our network.



### Network input

A major design decision is in which format the input audio signal will be fed to the network. 
Theoretically, raw array is possible

Creating a spectogram and feeding it as image

Using MFCC

Librosa is a python library to analyse audio signals. It provides several functions to extract features from audio data, which we will be using to preprocess the network input.

#hide 
TODO: Explain the different approaches to network input, architectures etc.

At this point we must decide in which format we will provide the network input. The dataset samples are given as raw audio data files, which could be directly used as input. However this has several disadvantages, one of them being the extensive memory usage. Loading the entire dataset into memory might exceed our capacities. A so-called data generator can provide relief. It loads the samples one by one in real-time during training and testing instead of using the naive approach of loading all samples at once. The code for such a data generator can be found at https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly. 

Still, computations on large data are more time-consuming, especially when they are loaded into memory on-the-fly like it is the case for a data generator. A better idea might be to extract features from the raw audio files and use these as input. This essentially compresses the input, which makes the use of a data generator unnecessary and greatly accelerates the training process.  

Therefore as a processing step, we are extracting so-called MFCC audio features from our input data using the Librosa library. The MFCC data are created one by one for each sample and stored in a variable X. This process is quite time-consuming. After it is done, we finally normalize the generated input data.

Now, we build our network model. 
The model starts with a convolutional layer followed by ReLU activation and a maxpool layer. Batch normalization is applied by inserting the corresponding layer before the activation function. 
This structure is repeated 3 more times, then the model ends with a fully connected layer of size 64 with batch normalization. The final output is given by a softmax layer which produces a probability distribution over the 41 classes.

As a loss function, we use cross entropy, and the Adam optimizer is used for training.

To prepare the output, we are converting the raw labels to integer indices and are setting the filename as the index for the train and test data:

## Results and Conclusion
Training for 30 epochs results in a promising training accuracy of around 90%. However the network currently shows strong overfitting, as the accuracy on the validation set barely reaches 40%. 
Further investigation is needed in order to find ways to reduce this massive gap. 

Future work includes exploring other ways to obtain information from sounds, for example predicting the distance from a sound source given a sound sample.

## References

{{ 'https://www.kaggle.com/fizzbuzz/beginner-s-guide-to-audio-data/' | fndetail: 1}}

{{ 'https://en.wikipedia.org/wiki/Pulse-code_modulation' | fndetail: 2}}